In [10]:
from etl_project.commons import utils
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import DateType
from pyspark.sql.window import Window
import random

In [2]:
shopping_data = \
[('Alex','2018-10-10','Paint',80),('Alex','2018-04-02','Ladder',20),('Alex','2018-06-22','Stool',20),\
('Alex','2018-12-09','Vacuum',40),('Alex','2018-07-12','Bucket',5),('Alex','2018-02-18','Gloves',5),\
('Alex','2018-03-03','Brushes',30),('Alex','2018-09-26','Sandpaper',10)]

In [3]:
session, log = utils.start_spark()

In [8]:
df = session.createDataFrame(shopping_data, ['name','date','product','price'])\
                .withColumn('date',F.col('date').cast(DateType()))

In [11]:
w0 = Window.partitionBy('name')

In [13]:
df.withColumn('price_rank',F.rank().over(w0.orderBy(F.col('price').asc()))).explain()

== Physical Plan ==
Window [rank(price#19L) windowspecdefinition(name#16, price#19L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS price_rank#66], [name#16], [price#19L ASC NULLS FIRST]
+- *(2) Sort [name#16 ASC NULLS FIRST, price#19L ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(name#16, 200), true, [id=#48]
      +- *(1) Project [name#16, cast(date#17 as date) AS date#24, product#18, price#19L]
         +- *(1) Scan ExistingRDD[name#16,date#17,product#18,price#19L]




In [ ]:
df.withColumn('price_rank',F.dense_rank().over(w0.orderBy(F.col('price').desc()))).show()

In [ ]:
df.withColumn('avg_to_date',     F.round(F.avg('price').over(w0.orderBy(F.col('date'))),2))\
  .withColumn('accumulating_sum',F.sum('price').over(w0.orderBy(F.col('date'))))\
  .withColumn('max_to_date',     F.max('price').over(w0.orderBy(F.col('date'))))\
  .withColumn('max_of_last2',    F.max('price').over(w0.orderBy(F.col('date')).rowsBetween(-1,Window.currentRow)))\
  .withColumn('items_to_date',   F.count('*').over(w0.orderBy(F.col('date'))))\
  .show()

In [ ]:
df.withColumn('days_from_last_purchase', F.datediff('date',F.lag('date',1).over(w0.orderBy(F.col('date')))))\
  .withColumn('days_before_next_purchase', F.datediff(F.lead('date',1).over(w0.orderBy(F.col('date'))),'date'))\
  .show()